# **Selenium**

**장점**
* table 형태가 아니더라도 내가 얻고 싶은 정보를 입맛대로 가져올 수 있다.
* 여러 페이지에서 정보를 가져올 수 있다.
* 로그인, 마우스 클릭, 캡쳐 등 동적으로 많은 자동화 업무를 할 수 있다.

**단점**
* 컴퓨터 사양에 따라 속도가 달라 원하는 정보를 추출하는데 시간이 걸릴 수도 있다.<br>
* 불러오는 속도와 python의 실행 속도가 맞지 않으면 오류가 난다.

## **세계 축구선수 Top200**


* **학습 목표**
    * 200위까지 선수명, 포지션, 나이, 국적, 소속, Market Value가 나타나는 데이터 프레임을 만들 수 있다.
    * 국적, 소속, 포지션 등을 통해 시각화하고, 데이터를 분석할 수 있다.

<div markdown="1" style="text-align:center; margin-bottom:10px">
    <img src="./images/soccer_html.png" width="50%">
</div>


## **1) Chrome 창 열기**

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

## **2) 사이트 이동**

In [23]:
# Chrome 열기
url = 'https://www.transfermarkt.com'
driver.get(url)

In [24]:
from selenium.webdriver.common.by import By  

privacy_iframe = 'sp_message_iframe_764226'
driver.switch_to.frame(privacy_iframe)

In [25]:
privacy_css = '#notice > div.message-component.message-row.mobile-reverse > div:nth-child(2) > button'
driver.find_element(By.CSS_SELECTOR, privacy_css).click()

In [26]:
driver.switch_to.default_content()

In [33]:
# onclick인 경우
players_css = '#main > header > nav > ul > li:nth-child(4) > div > div:nth-child(2) > ul > li:nth-child(2) > a'
players_url = driver.find_element(By.CSS_SELECTOR, players_css)
driver.execute_script('arguments[0].click();', players_url)

In [85]:
import re
import pandas as pd
import time

rank_n = 100
cnt = 0
page = 1
rows = []

print('Start | ', end='')
while True:
    print(f'{page}페이지 > ', end='')
    players = driver.find_elements(By.CSS_SELECTOR, 'table.items > tbody > tr')

    for player in players:
        rank = player.find_element(By.CSS_SELECTOR, f'td:nth-child({1})').text
        name, position = player.find_element(By.CSS_SELECTOR, f'td:nth-child({2})').text.split('\n')
        age = player.find_element(By.CSS_SELECTOR, f'td:nth-child({3})').text
        nat_sel = player.find_elements(By.CSS_SELECTOR, f'td:nth-child({4}) > img')
        nationality = ', '.join([nat.get_attribute('title') for nat in nat_sel])
        club = player.find_element(By.CSS_SELECTOR, f'td:nth-child({5}) > a').get_attribute('title')
        value_text = player.find_element(By.CSS_SELECTOR, f'td:nth-child({6}) > a').text
        value = re.sub('[^0-9.]','',value_text)

        cnt += 1
        rows.append([rank, name, position, nationality, club, value])

        if cnt >= rank_n:
            break
    
    if cnt < rank_n:
        page += 1
        page_css = f'#yw1 > div.pager > ul > li:nth-child({page}) > a'
        driver.find_element(By.CSS_SELECTOR, page_css).click()
        time.sleep(3)
    else:
        break
print('End')
pd.DataFrame(rows)

Start | 1페이지 > 2페이지 > 3페이지 > 4페이지 > End


,0,1,2,3,4,5
0,1,Kylian Mbappé,Centre-Forward,"France, Cameroon",Paris Saint-Germain,180.00
1,2,Erling Haaland,Centre-Forward,Norway,Manchester City,170.00
2,3,Jude Bellingham,Central Midfield,England,Borussia Dortmund,120.00
3,4,Vinicius Junior,Left Winger,"Brazil, Spain",Real Madrid,120.00
4,5,Jamal Musiala,Attacking Midfield,"Germany, England",Bayern Munich,110.00
...,...,...,...,...,...,...
95,96,Gianluigi Donnarumma,Goalkeeper,Italy,Paris Saint-Germain,50.00
96,97,Lucas Hernández,Centre-Back,"France, Spain",Bayern Munich,50.00
97,98,Ivan Toney,Centre-Forward,"England, Jamaica",Brentford FC,50.00
98,99,Marco Verratti,Central Midfield,Italy,Paris Saint-Germain,50.00


## **2) 정보 추출**

In [27]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')
soup.prettify()

'<!DOCTYPE html>\n<html lang="ko">\n <head>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n  <title>\n   멜론차트&gt;TOP100&gt;멜론\n  </title>\n  <meta content="음악서비스, 멜론차트, 멜론TOP100, 최신음악, 인기가요, 뮤직비디오, 앨범, 플레이어, 스트리밍, 다운로드, 아티스트플러스, 아티스트채널" name="keywords"/>\n  <meta content="No.1 뮤직플랫폼 멜론! 최신 트렌드부터 나를 아는 똑똑한 음악추천까지!" name="description"/>\n  <meta content="ee85ff6db1fa8f2226bcb671ecb2bcdbcffb6f8b" name="naver-site-verification"/>\n  <meta content="q4tbTQhmxa4La3OdNLsNOCxrJ_WV6lUlBFrFW4-HqQc" name="google-site-verification"/>\n  <meta content="357952407588971" property="fb:app_id"/>\n  <meta content="Melon" property="og:title"/>\n  <meta content="https://cdnimg.melon.co.kr/resource/image/web/common/logo_melon142x99.png" property="og:image"/>\n  <meta content="음악이 필요한 순간, 멜론" property="og:description"/>\n  <meta content="http://www.melon.com/chart/index.htm" property="og:url"/>\n  <meta content="webs

<div markdown="1" style="justify-content:center; align-items:center; margin-bottom:10px;">
<img src="./images/beautifulsoup01.png" width="20%">
<img src="./images/beautifulsoup02.png" width="20%" style="margin-left:20px">
</div> 

* 웹 페이지에서 곡 리스트 테이블은 class가 'lst50`과 'lst100'으로 나뉘어져 있다. 
* 필요한 정보     

    <table>
        <tr> <th>필요한 정보</th><th>태그/id/class</th> </tr>
        <tr> <td>순위</th><td>class="rank"</td> </tr>
        <tr> <td>곡 명</th><td>class="ellipsis rank01"</td> </tr>
        <tr> <td>가수 명</th><td>class="ellipsis rank02"</td> </tr>
    </table>

In [74]:
import pandas as pd
play_lst = []
for lst in ['lst50', 'lst100']:
    rows = soup.find_all('tr', class_=lst)
    for row in rows:
        rank = row.find(class_='rank').text
        song = row.find(class_='ellipsis rank01').text.strip()
        singer = row.find(class_='ellipsis rank02').span.text.strip()
        play_lst.append([rank, song, singer])

day = soup.find(class_='year').text + ' ' + soup.find(class_='hour').text
print(day, '기준 Melon 차트')
melon = pd.DataFrame(play_lst, columns=['순위', '곡 명', '가수 명'])
melon.set_index('순위', inplace=True)
melon

2023.04.12 17:00 기준 Melon 차트


,곡 명,가수 명
순위,,
1,Kitsch,IVE (아이브)
2,I AM,IVE (아이브)
3,꽃,지수 (JISOO)
4,Ditto,NewJeans
5,Hype boy,NewJeans
...,...,...
96,그중에 그대를 만나,김호중
97,That’s Not How This Works (feat. Dan + Shay),Charlie Puth
98,딱 10CM만,"10CM, BIG Naughty (서동현)"
